# Along-slope velocity

Calculate the velocity component tangent to the a bathymetry contour.

Load modules

In [19]:
from pathlib import Path
# import cosima_cookbook as cc
from dask.distributed import Client
import numpy as np
import xarray as xr

import xgcm
import cf_xarray

# For plotting
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cmocean as cm
import pyproj

import intake
cat = intake.cat.access_nri
cat

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{seaIce, ocean}","{1day, 3hr, fx, 1mon, 3mon}","{total_ocean_lw_heat, sfc_hflux_coupler, vvel_m, ULON, total_ocean_evap, temp_global_ave, TLON, evap, total_ocean_melt, pot_rho_1, sea_level, temp_vdiffuse_impl, uhrho_et, sfc_salt_flux_ice, wt, a..."
01deg_jra55v140_iaf,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1mon, 1day, fx}","{total_ocean_lw_heat, diff_cbt_t, sfc_hflux_coupler, vvel_m, ULON, total_ocean_evap, temp_global_ave, mld_max, TLON, evap, total_ocean_melt, sea_level, sfc_salt_flux_ice, wt, average_DT, NCAT, tot..."
01deg_jra55v140_iaf_cycle2,{ACCESS-OM2},{Cycle 2 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1mon, 1day, fx}","{total_ocean_lw_heat, diff_cbt_t, Tinz, sfc_hflux_coupler, daidtd, ULON, vvel_m, total_ocean_evap, temp_global_ave, mld_max, TLON, evap, total_ocean_melt, sea_level, sfc_salt_flux_ice, wt, average..."
01deg_jra55v140_iaf_cycle3,{ACCESS-OM2},{Cycle 3 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1mon, 1day, fx}","{total_ocean_lw_heat, diff_cbt_t, Tinz, sfc_hflux_coupler, daidtd, ULON, vvel_m, total_ocean_evap, temp_global_ave, mld_max, TLON, evap, total_ocean_melt, sea_level, sfc_salt_flux_ice, uhrho_et, w..."
01deg_jra55v140_iaf_cycle4,{ACCESS-OM2},{Cycle 4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1day, 3hr, fx, 1mon, 6hr}","{ULON, temp_global_ave, TLON, total_ocean_melt, sfc_salt_flux_ice, surface_o2, fhocn_ai_m, stf09, alidr_ai, dic_yflux_adv, o2_xflux_adv, total_ocean_hflux_prec, flatn_ai_m, algal_N_m, river, dic, ..."
01deg_jra55v140_iaf_cycle4_jra55v150_extension,{ACCESS-OM2},{Extensions of cycle 4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 interannual forcing},"{seaIce, ocean}","{1mon, subhr, 1day, fx}","{total_ocean_lw_heat, diff_cbt_t, sfc_hflux_coupler, daidtd, ULON, vvel_m, fN_ai_m, total_ocean_evap, temp_global_ave, mld_max, radbio_int100, TLON, evap, src06, total_ocean_melt, sea_level, sfc_s..."
01deg_jra55v150_iaf_cycle1,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1mon, 1day, fx}","{diff_cbt_t, sfc_hflux_coupler, ULON, mld_max, TLON, evap, sea_level, sfc_salt_flux_ice, average_DT, time_bnds, swflx, tx_trans, sfc_hflux_pme, runoff, lw_heat, sfc_hflux_from_runoff, kmt, mh_flux..."
025deg_era5_iaf,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with ERA5 interannual\nforcing (1980-2021)},"{seaIce, ocean}","{1mon, 1day, fx}","{total_ocean_lw_heat, sfc_hflux_coupler, vvel_m, ULON, ice_present_m, total_ocean_evap, strinty_m, temp_global_ave, mld_max, TLON, evap, total_ocean_melt, sea_level, sfc_salt_flux_ice, fswabs_ai_m..."
025deg_era5_ryf,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with ERA5 RYF9091 repeat\nyear forcing (May 1990 to Apr 1991)},"{seaIce, ocean}","{1mon, 1day, fx}","{total_ocean_lw_heat, diff_cbt_t, sfc_hflux_coupler, vvel_m, ULON, ice_present_m, total_ocean_evap, strinty_m, temp_global_ave, mld_max, TLON, evap, total_ocean_melt, sea_level, sfc_salt_flux_ice,..."


By default retain metadata after operations. This can retain out of date metadata, so some caution is required

In [20]:
xr.set_options(keep_attrs=True)

Start a cluster with multiple cores

In [21]:
client = Client(threads_per_worker = 1)
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38713 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:38713/status,
Dashboard: http://127.0.0.1:38713/status,Workers: 4
Total threads: 4,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45875,Workers: 4
Dashboard: http://127.0.0.1:38713/status,Total threads: 4
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:40735,Total threads: 1
Dashboard: /proxy/42553/status,Memory: 31.30 GiB
Nanny: tcp://127.0.0.1:39029,


### Load Data

Nominate a database from which to load the data and define an experiment

In [22]:
experiment = '01deg_jra55v13_ryf9091'

Limit to Southern Ocean and single RYF year

In [23]:
lat_slice  = slice(-80, -59)

start_time = '2086-01-01'
end_time   = '2086-12-31'
time_slice = slice(start_time, end_time)

Load bathymetry data. Discard the geolon and geolat coordinates: these are 2D curvilinear coordinates that are only required when working above 65N

In [24]:
# hu = cc.querying.getvar(experiment, 'hu', session, n=1).drop(['geolat_c', 'geolon_c']).sel(yu_ocean=lat_slice).load()
exp = cat[experiment]
hu = exp.search(variable="hu", filename="ocean_grid.nc", path=".*output000/ocean.*")  # FIXME hack to be removed when intake updated
hu = hu.to_dask()
hu

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'cfradial1' loading failed:
cannot import name 'HybridMappingProxy' from 'xarray.core.utils' (/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/core/utils.py)
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'datamet' loading failed:
cannot import name 'HybridMappingProxy' from 'xarray.core.utils' (/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/core/utils.py)
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'furuno' loading failed:
can

<xarray.Dataset> Size: 117MB
Dimensions:   (yu_ocean: 2700, xu_ocean: 3600)
Coordinates:
  * xu_ocean  (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 22kB -81.09 -81.05 -81.0 ... 89.92 89.96 90.0
    geolon_c  (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    geolat_c  (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
Data variables:
    hu        (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
Attributes: (12/19)
    filename:                                 ocean_grid.nc
    title:                                    ACCESS-OM2-01
    grid_type:                                mosaic
    grid_tile:                                1
    intake_esm_vars:                          ['hu']
    intake_esm_attrs:path:                    /g/data/ik11/outputs/access-om2...
    ...                                       ...
    intake_esm_attrs:variable_cell_methods:   time: point,time: point,time: p...
    intake_esm_attrs:variable_units:          degrees_E,degrees_N,degrees_E,d...
    intake_esm_attrs:filename:                ocean_grid.nc
    intake_esm_attrs:file_id:                 ocean_grid
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   ocean_grid.fx

Load velocity data, limit to upper 500m and take the mean in time

In [25]:
uv = exp.search(variable=["u", "v"], filename="ocean.nc", frequency="1mon")
uv = uv.to_dask().sel(yu_ocean=lat_slice, st_ocean=slice(0, 500))
uv.chunk(chunks={"time": -1, "st_ocean": 1, "yu_ocean": 484//4, "xu_ocean": 72})
uvm = uv.mean("time")
uvm

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'cfradial1' loading failed:
cannot import name 'HybridMappingProxy' from 'xarray.core.utils' (/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/core/utils.py)
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'cfradial1' loading failed:
cannot import name 'HybridMappingProxy' from 'xarray.core.utils' (/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/core/utils.py)
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'datamet' loading failed:


<xarray.Dataset> Size: 544MB
Dimensions:   (st_ocean: 39, yu_ocean: 484, xu_ocean: 3600)
Coordinates:
  * st_ocean  (st_ocean) float64 312B 0.5413 1.681 2.94 ... 383.0 423.7 468.4
  * xu_ocean  (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 4kB -79.99 -79.95 -79.9 ... -59.06 -59.01
Data variables:
    v         (st_ocean, yu_ocean, xu_ocean) float32 272MB dask.array<chunksize=(7, 274, 400), meta=np.ndarray>
    u         (st_ocean, yu_ocean, xu_ocean) float32 272MB dask.array<chunksize=(7, 274, 400), meta=np.ndarray>
Attributes:
    filename:                        ocean.nc
    title:                           ACCESS-OM2-01
    grid_type:                       mosaic
    grid_tile:                       1
    intake_esm_vars:                 ['v', 'u']
    intake_esm_attrs:realm:          ocean
    intake_esm_attrs:frequency:      1mon
    intake_esm_attrs:filename:       ocean.nc
    intake_esm_attrs:file_id:        ocean
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          ocean.1mon

Load model grid information directly from a grid data file

In [26]:
path_to_folder = Path('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/')
grid = xr.open_mfdataset(path_to_folder / 'ocean_grid.nc', combine='by_coords').drop(['geolon_t', 'geolat_t', 'geolon_c', 'geolat_c'])
grid

/scratch/tm70/mcw120/tmp/ipykernel_1330101/2240020569.py:2: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  grid = xr.open_mfdataset(path_to_folder / 'ocean_grid.nc', combine='by_coords').drop(['geolon_t', 'geolat_t', 'geolon_c', 'geolat_c'])


<xarray.Dataset> Size: 428MB
Dimensions:     (xt_ocean: 3600, yt_ocean: 2700, time: 1, xu_ocean: 3600,
                 yu_ocean: 2700)
Coordinates:
  * xt_ocean    (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean    (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
  * time        (time) object 8B 1900-04-01 00:00:00
  * xu_ocean    (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean    (yu_ocean) float64 22kB -81.09 -81.05 -81.0 ... 89.92 89.96 90.0
Data variables:
    ht          (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    hu          (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dxt         (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dyt         (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dxu         (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dyu         (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    area_t      (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    area_u      (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    kmt         (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    kmu         (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    drag_coeff  (time, yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
Attributes:
    filename:   ocean_grid.nc
    title:      ACCESS-OM2-01
    grid_type:  mosaic
    grid_tile:  1

### Along-slope velocity

We calculate the along-slope velocity component by projecting the velocity field to the tangent vector, $u_{along} = \boldsymbol{u \cdot \hat{t}}$, and the cross-slope component by projecting to the normal vector, $v_{cross} = \boldsymbol{u \cdot \hat{n}}$. The schematic below defines the unit normal normal and tangent vectors for a given bathymetric contour, $\boldsymbol{n}$ and $\boldsymbol{t}$ respectively. 

![Sketch of topographic gradient](images/topographic_gradient_sketch.png)

Accordingly, the code below calculates the along-slope velocity component as

$$ u_{along} = (u,v) \boldsymbol{\cdot} \left(\frac{h_y}{|\nabla h|} , -\frac{h_x}{|\nabla h|}\right) = 
u \frac{h_y}{|\nabla h|} - v \frac{h_x}{|\nabla h|}, $$  

and similarly the cross-slope velocity component as

$$ v_{cross} = (u,v) \boldsymbol{\cdot} \left(\frac{h_x}{|\nabla h|} , \frac{h_y}{|\nabla h|}\right)  = 
u \frac{h_x}{|\nabla h|} + v \frac{h_y}{|\nabla h|}.$$ 


We need the derivatives of the bathymetry which we compute using the `xgcm` functionality.

In [27]:
# Give information on the grid: location of u (momentum) and t (tracer) points on B-grid 
ds = xr.merge([hu, grid])
ds.coords['xt_ocean'].attrs.update(axis='X')
ds.coords['xu_ocean'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds.coords['yt_ocean'].attrs.update(axis='Y')
ds.coords['yu_ocean'].attrs.update(axis='Y', c_grid_axis_shift=0.5)

grid = xgcm.Grid(ds, periodic=['X'])

# Take topographic gradient (simple gradient over one grid cell) and move back to u-grid
dhu_dx = grid.interp( grid.diff(ds.hu, 'X') / grid.interp(ds.dxu, 'X'), 'X')

# In meridional direction, we need to specify what happens at the boundary
dhu_dy = grid.interp( grid.diff(ds.hu, 'Y', boundary='extend') / grid.interp(ds.dyt, 'X'), 'Y', boundary='extend')

# Select latitude slice
dhu_dx = dhu_dx.sel(yu_ocean=lat_slice)
dhu_dy = dhu_dy.sel(yu_ocean=lat_slice)

# Magnitude of the topographic slope (to normalise the topographic gradient)
topographic_slope_magnitude = np.sqrt(dhu_dx**2 + dhu_dy**2)

ds

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return

<xarray.Dataset> Size: 506MB
Dimensions:     (yu_ocean: 2700, xu_ocean: 3600, xt_ocean: 3600,
                 yt_ocean: 2700, time: 1)
Coordinates:
  * xu_ocean    (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean    (yu_ocean) float64 22kB -81.09 -81.05 -81.0 ... 89.92 89.96 90.0
    geolon_c    (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    geolat_c    (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
  * xt_ocean    (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean    (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
  * time        (time) object 8B 1900-04-01 00:00:00
Data variables:
    hu          (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    ht          (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dxt         (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dyt         (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dxu         (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dyu         (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    area_t      (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    area_u      (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    kmt         (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    kmu         (yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    drag_coeff  (time, yu_ocean, xu_ocean) float32 39MB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
Attributes: (12/19)
    filename:                                 ocean_grid.nc
    title:                                    ACCESS-OM2-01
    grid_type:                                mosaic
    grid_tile:                                1
    intake_esm_vars:                          ['hu']
    intake_esm_attrs:path:                    /g/data/ik11/outputs/access-om2...
    ...                                       ...
    intake_esm_attrs:variable_cell_methods:   time: point,time: point,time: p...
    intake_esm_attrs:variable_units:          degrees_E,degrees_N,degrees_E,d...
    intake_esm_attrs:filename:                ocean_grid.nc
    intake_esm_attrs:file_id:                 ocean_grid
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   ocean_grid.fx

Calculate along-slope velocity component

In [12]:
# Along-slope velocity
alongslope_velocity = uv.u * dhu_dy / topographic_slope_magnitude - uv.v * dhu_dx / topographic_slope_magnitude

# Load the data
# alongslope_velocity = alongslope_velocity.load()
# warnings might come up in points where we divide by NaN/0,
# i.e., when there is no topographic gradient and warning can be ignored
alongslope_velocity.chunk(chunks={"time": -1, "st_ocean": 1, "yu_ocean": 484//4, "xu_ocean": 72})
alongslope_velocity

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
 

Vertical averaging (we only need this to plot the velocity on a map)

In [ ]:
# Import edges of st_ocean and add lat/lon dimensions:
# st_edges_ocean = cc.querying.getvar(experiment, 'st_edges_ocean', session,
#                                     start_time=start_time, end_time=end_time, n=1)
cat.search(variable='st_edges_ocean')  # FIXME - this variable doesn't exist!
# st_edges_ocean = exp.search(variable="st_edges_ocean")
# st_edges_ocean
# st_edges_array = st_edges_ocean.expand_dims({'yu_ocean': u.yu_ocean, 'xu_ocean': u.xu_ocean}, axis=[1, 2])

,model,description,realm,frequency,variable
name,,,,,


In [ ]:
# Adjust edges at bottom for partial thickness:
st_edges_with_partial = st_edges_array.where(st_edges_array<hu, other=hu)
thickness = st_edges_with_partial.diff(dim='st_edges_ocean')

# Change coordinate of thickness to st_ocean (needed for multipling with other variables):
st_ocean = cc.querying.getvar(experiment, 'st_ocean', session, n=1)
thickness['st_edges_ocean'] = st_ocean.values
thickness = thickness.rename(({'st_edges_ocean': 'st_ocean'}))
thickness = thickness.sel(st_ocean=slice(0, 500))

# Depth average gives us the barotropic velocity
barotropic_alongslope_velocity = (alongslope_velocity * thickness).sum('st_ocean') / thickness.sum('st_ocean')

### Plotting

Create a circular path to clip plots

In [ ]:
theta  = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.45
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

Create a land mask for plotting, set land cells to 1 and rest to NaN

In [ ]:
land = xr.where(np.isnan(hu.rename('land')), 1, np.nan)

#### Map of along-slope velocity with bathymetry contours

In [ ]:
fig = plt.figure(1, figsize=(15, 15))

ax = plt.subplot(1, 1, 1, projection=ccrs.SouthPolarStereo(), facecolor="darkgrey")
ax.set_boundary(circle, transform=ax.transAxes)
    
# Filled land 
land.plot.contourf(ax=ax, colors='darkgrey',
                   zorder=2,
                   transform=ccrs.PlateCarree(),
                   add_colorbar=False)

# Coastline
land.fillna(0).plot.contour(ax=ax, colors='k', levels=[0, 1],
                            transform=ccrs.PlateCarree(),
                            add_colorbar=False)

# Depth contours
hu.plot.contour(ax=ax, levels=[500, 1000, 2000, 3000],
                colors='0.2', linewidths=[0.5, 2, 0.5, 0.5], alpha=0.5,
                transform=ccrs.PlateCarree())

# Along slope barotropic velocity
sc = barotropic_alongslope_velocity.plot(ax=ax, cmap=cm.cm.curl,
                                         transform=ccrs.PlateCarree(),
                                         vmin=-0.3, vmax=0.3,
                                         cbar_kwargs={'orientation': 'vertical',
                                                      'shrink': 0.25,
                                                      'extend': 'both',
                                                      'label': None,
                                                      'aspect': 8})
  
ax.set_title('Along-slope barotropic velocity (m s$^{-1}$)');

In [ ]:
client.close()